### Rede Neural

In [3]:
import time
import numpy as np
from random import randint
from typing import List, Callable

class NN:
    """
    Uma classe para representar uma rede neural simples com uma camada oculta.
    """
    def __init__(self, inputs: int, hidden_units: int, outputs: int, activation: str = 'sigmoid'):
        """
        Inicializa a rede neural com pesos e biases aleatórios.

        Args:
            inputs (int): Número de neurônios na camada de entrada.
            hidden_units (int): Número de neurônios na camada oculta.
            outputs (int): Número de neurônios na camada de saída.
            activation (str): Nome da função de ativação ('sigmoid', 'tanh', 'relu').
        """
        # Arquitetura da rede
        self.inputs = inputs
        self.hidden_units = hidden_units
        self.outputs = outputs
        self.activation_str = activation

        # Parâmetros da rede (pesos e biases)
        # Camada 1: Entrada -> Oculta
        self.W1 = np.random.randn(hidden_units, inputs)  # Pesos
        self.b1 = np.random.randn(1, hidden_units)            # Biases

        # Camada 2: Oculta -> Saída
        self.W2 = np.random.randn(outputs, hidden_units) # Pesos
        self.b2 = np.random.randn(1, outputs)                 # Biases
        
        # Função de ativação
        self.actv = self._get_activation_function(activation)

    def _get_activation_function(self, name: str) -> Callable[[np.ndarray], np.ndarray]:
        """Retorna a função de ativação correspondente ao nome."""
        match name:
            case 'sigmoid':
                return lambda x: 1 / (1 + np.exp(-x))
            case 'tanh':
                return np.tanh
            case 'relu':
                return lambda x: np.maximum(0, x)
            case _:
                raise ValueError(f"Função de ativação '{name}' não suportada.")

    def predict(self, X: np.ndarray) -> np.ndarray:
        """
        Realiza a passagem para a frente (forward pass) na rede para fazer uma predição.
        """
        # Camada oculta
        z1 = X @ self.W1.T + self.b1
        a1 = self.actv(z1)
        
        # Camada de saída
        z2 = a1 @ self.W2.T + self.b2
        # A camada de saída não terá ativação neste exemplo (comum para regressão)
        output = z2
        
        return output

    def __repr__(self):
        """Representação em string do objeto NN."""
        return (f"NN(inputs={self.inputs}, hidden_units={self.hidden_units}, "
                f"outputs={self.outputs}, activation='{self.activation_str}')")


# --- 2. Funções para Achatar e Reconstruir os Parâmetros ---

def flatten_parameters(nn: NN) -> np.ndarray:
    """
    Pega uma rede neural (NN) e coloca todos os seus parâmetros (pesos e biases)
    alinhados em um único array numpy.

    Args:
        nn (NN): A rede neural a ser "achatada".

    Returns:
        np.ndarray: Um array 1D contendo todos os parâmetros da rede.
    """
    # Coleta todos os arrays de parâmetros
    params = [nn.W1, nn.b1, nn.W2, nn.b2]
    
    # Usa np.concatenate com uma lista de parâmetros achatados (.ravel())
    flat_params = np.concatenate([p.ravel() for p in params])
    
    return flat_params

def unflatten_parameters(nn: NN, flat_params: np.ndarray):
    """
    Pega um array 1D de parâmetros e os usa para redefinir os pesos e biases
    de uma rede neural (NN) existente.

    Args:
        nn (NN): A rede neural cujos parâmetros serão atualizados.
        flat_params (np.ndarray): O array 1D com os novos parâmetros.
    """
    # Calcula o número de elementos em cada matriz de parâmetros
    s_w1 = nn.W1.size
    s_b1 = nn.b1.size
    s_w2 = nn.W2.size
    s_b2 = nn.b2.size
    
    # Verifica se o tamanho do array de entrada corresponde ao total de parâmetros
    total_params = s_w1 + s_b1 + s_w2 + s_b2
    if flat_params.size != total_params:
        raise ValueError(f"Tamanho do array incorreto. Esperado: {total_params}, Recebido: {flat_params.size}")

    # Ponteiro para marcar a posição atual no array achatado
    pointer = 0
    
    # Extrai, remodela e atribui os parâmetros para a Camada 1
    nn.W1 = flat_params[pointer : pointer + s_w1].reshape(nn.W1.shape)
    pointer += s_w1
    nn.b1 = flat_params[pointer : pointer + s_b1].reshape(nn.b1.shape)
    pointer += s_b1
    
    # Extrai, remodela e atribui os parâmetros para a Camada 2
    nn.W2 = flat_params[pointer : pointer + s_w2].reshape(nn.W2.shape)
    pointer += s_w2
    nn.b2 = flat_params[pointer : pointer + s_b2].reshape(nn.b2.shape)

#### Teste NN

In [4]:
if __name__ == "__main__":
    # Definição da arquitetura da rede
    INPUTS = 2
    HIDDEN_UNITS = 4
    OUTPUT = 1
    ACTIVATION = 'sigmoid'

    # 1. Criar a primeira rede neural (nn1)
    print("--- Criando a Rede Neural 1 (nn1) ---")
    nn1 = NN(inputs=INPUTS, hidden_units=HIDDEN_UNITS, outputs=OUTPUT, activation=ACTIVATION)
    print(f"Rede criada: {nn1}")
    
    # Criar um dado de entrada de exemplo
    input_data = np.random.normal(0, 1, (1, INPUTS))
    print(f"\nEntrada de exemplo (shape {input_data.shape}):\n{input_data}")

    # Fazer uma predição com a nn1
    output1 = nn1.predict(input_data)
    print(f"\nSaída da nn1 (com pesos originais): {output1.item():.4f}")

    # 2. Achatando os parâmetros da nn1
    print("\n--- Achatando parâmetros da nn1 ---")
    flat_array = flatten_parameters(nn1)
    print(f"Array de parâmetros achatado (shape {flat_array.shape}):")
    print(flat_array)

    # 3. Criar uma segunda rede neural (nn2), com pesos e biases diferentes
    print("\n--- Criando a Rede Neural 2 (nn2) com pesos diferentes ---")
    nn2 = NN(inputs=INPUTS, hidden_units=HIDDEN_UNITS, outputs=OUTPUT, activation=ACTIVATION)
    
    # Fazer uma predição com a nn2 para mostrar que o resultado é diferente
    output2_antes = nn2.predict(input_data)
    print(f"Saída da nn2 (antes da cópia): {output2_antes.item():.4f}")

    # 4. Reconstruir a nn2 usando os parâmetros achatados da nn1
    print("\n--- Reconstruindo nn2 com os parâmetros da nn1 ---")
    unflatten_parameters(nn2, flat_array)
    print("Parâmetros de nn1 foram copiados para nn2.")

    # 5. Verificar se as saídas agora são idênticas
    print("\n--- Verificando os resultados ---")
    output2_depois = nn2.predict(input_data)
    print(f"Saída da nn1 original:          {output1.item():.8f}")
    print(f"Saída da nn2 após reconstrução: {output2_depois.item():.8f}")

--- Criando a Rede Neural 1 (nn1) ---
Rede criada: NN(inputs=2, hidden_units=4, outputs=1, activation='sigmoid')

Entrada de exemplo (shape (1, 2)):
[[1.76530781 1.05662411]]

Saída da nn1 (com pesos originais): -1.8337

--- Achatando parâmetros da nn1 ---
Array de parâmetros achatado (shape (17,)):
[ 0.75878478 -0.84025675  0.08790717  1.25689885 -1.16550726  1.46611709
 -0.21890065  1.26656662 -0.78666299  0.00507219  0.56333208  0.37435207
  0.51868415 -0.3149266   0.11273953 -2.07770341 -0.20920101]

--- Criando a Rede Neural 2 (nn2) com pesos diferentes ---
Saída da nn2 (antes da cópia): 0.5184

--- Reconstruindo nn2 com os parâmetros da nn1 ---
Parâmetros de nn1 foram copiados para nn2.

--- Verificando os resultados ---
Saída da nn1 original:          -1.83373285
Saída da nn2 após reconstrução: -1.83373285


In [12]:
import time
import numpy as np
import random
from typing import List, Callable, Optional, Tuple
import math
import pygame

# ==============================================================================
# 1. CÓDIGO DA REDE NEURAL (FORNECIDO)
# ==============================================================================

class NN:
    """
    Uma classe para representar uma rede neural simples com uma camada oculta.
    """
    def __init__(self, inputs: int, hidden_units: int, outputs: int, activation: str = 'tanh'):
        """
        Inicializa a rede neural com pesos e biases aleatórios.
        """
        self.inputs = inputs
        self.hidden_units = hidden_units
        self.outputs = outputs
        self.activation_str = activation
        # Inicialização de pesos com He/Xavier para tanh/relu é geralmente melhor
        self.W1 = np.random.randn(hidden_units, inputs) * np.sqrt(2. / inputs)
        self.b1 = np.zeros((1, hidden_units))
        self.W2 = np.random.randn(outputs, hidden_units) * np.sqrt(2. / hidden_units)
        self.b2 = np.zeros((1, outputs))
        self.actv = self._get_activation_function(activation)

    def _get_activation_function(self, name: str) -> Callable[[np.ndarray], np.ndarray]:
        match name:
            case 'sigmoid': return lambda x: 1 / (1 + np.exp(-x))
            case 'tanh': return np.tanh
            case 'relu': return lambda x: np.maximum(0, x)
            case _: raise ValueError(f"Função de ativação '{name}' não suportada.")

    def predict(self, X: np.ndarray) -> np.ndarray:
        if X.ndim == 1: X = X.reshape(1, -1)
        z1 = X @ self.W1.T + self.b1
        a1 = self.actv(z1)
        z2 = a1 @ self.W2.T + self.b2
        # Usar tanh na saída para manter as ações entre -1 e 1
        output = np.tanh(z2) 
        return output

    def __repr__(self):
        return (f"NN(inputs={self.inputs}, hidden_units={self.hidden_units}, "
                f"outputs={self.outputs}, activation='{self.activation_str}')")

# Funções auxiliares para achatamento/reconstrução de parâmetros
def flatten_parameters(nn: NN) -> np.ndarray:
    params = [nn.W1, nn.b1, nn.W2, nn.b2]
    return np.concatenate([p.ravel() for p in params])

def unflatten_parameters(nn: NN, flat_params: np.ndarray):
    s_w1, s_b1, s_w2, s_b2 = nn.W1.size, nn.b1.size, nn.W2.size, nn.b2.size
    total_params = s_w1 + s_b1 + s_w2 + s_b2
    if flat_params.size != total_params:
        raise ValueError(f"Tamanho do array incorreto. Esperado: {total_params}, Recebido: {flat_params.size}")
    pointer = 0
    nn.W1 = flat_params[pointer : pointer + s_w1].reshape(nn.W1.shape); pointer += s_w1
    nn.b1 = flat_params[pointer : pointer + s_b1].reshape(nn.b1.shape); pointer += s_b1
    nn.W2 = flat_params[pointer : pointer + s_w2].reshape(nn.W2.shape); pointer += s_w2
    nn.b2 = flat_params[pointer : pointer + s_b2].reshape(nn.b2.shape)

# ==============================================================================
# 2. CLASSES DO JOGO
# ==============================================================================

# Constantes de configuração
AGENT_SIZE = 12
FRUIT_SIZE = 5
MAX_LIFE = 1000
MAX_STAMINA = 500
LIFE_DECAY_RATE = 30  # Pontos de vida perdidos por segundo (multiplicado por 5 no update)
LIFE_GAIN_ON_EAT = 250
STAMINA_COST_MOVE = 30 # Custo de estamina por segundo de movimento
STAMINA_DEPLETION_SLOW_FACTOR = 0.1 # Fator de lentidão quando estamina = 0
STAMINA_DEPLETION_COOLDOWN_DURATION = 5.0 # Segundos que o agente fica lento após esgotar estamina

FIELD_OF_VIEW_ANGLE = np.deg2rad(30) # 45 graus de campo de visão
VISION_RANGE = 150 # Quão longe o agente pode "ver"

# Cores e transparência para renderização
AGENT_ACTIVE_COLOR = (200, 200, 250) # Azul claro
AGENT_DEAD_COLOR = (80, 80, 80)      # Cinza escuro
FOV_COLOR = (50, 50, 255)            # Azul para FOV
FOV_ALPHA = 40                       # Transparência para FOV
FRUIT_COLOR = (50, 200, 50)          # Verde para fruta
FRUIT_ALPHA = 150                    # Transparência para fruta

class Fruit:
    def __init__(self, world_size: Tuple[int, int]):
        self.world_size = world_size
        self.pos = pygame.math.Vector2(
            random.randint(FRUIT_SIZE, world_size[0] - FRUIT_SIZE),
            random.randint(FRUIT_SIZE, world_size[1] - FRUIT_SIZE)
        )
        # Superfície temporária para desenho com transparência da fruta
        self.transparent_surface = pygame.Surface((FRUIT_SIZE * 2, FRUIT_SIZE * 2), pygame.SRCALPHA)

    def respawn(self):
        self.pos.x = random.randint(FRUIT_SIZE, self.world_size[0] - FRUIT_SIZE)
        self.pos.y = random.randint(FRUIT_SIZE, self.world_size[1] - FRUIT_SIZE)

    def draw(self, surface: pygame.Surface, is_semi_transparent: bool):
        if is_semi_transparent: # Se a renderização semi-transparente está ativa, as frutas também serão
            self.transparent_surface.fill((0, 0, 0, 0)) # Limpa com cor transparente
            local_center = pygame.math.Vector2(FRUIT_SIZE, FRUIT_SIZE)
            pygame.draw.circle(self.transparent_surface, FRUIT_COLOR + (FRUIT_ALPHA,), local_center, FRUIT_SIZE)
            surface.blit(self.transparent_surface, self.pos - local_center)
        else:
            pygame.draw.circle(surface, FRUIT_COLOR, self.pos, FRUIT_SIZE)

class Agent:
    def __init__(self, nn: NN, world_size: Tuple[int, int], use_stamina: bool):
        self.nn = nn
        self.world_size = world_size
        self.use_stamina = use_stamina

        self.pos = pygame.math.Vector2(
            random.randint(0, world_size[0]),
            random.randint(0, world_size[1])
        )
        self.angle = random.uniform(0, 2 * math.pi)
        self.base_speed = 120  # pixels por segundo (velocidade máxima)
        self.rotation_speed = 3.5  # radianos por segundo

        self.life = MAX_LIFE
        self.stamina = MAX_STAMINA if use_stamina else 0

        self.is_alive = True
        self.lifespan = 0.0  # Tempo que o agente viveu

        # NOVAS VARIÁVEIS PARA LÓGICA DE ESTAMINA
        self.is_stamina_depleted = False # Verdadeiro quando a estamina chegou a zero e o cooldown está ativo
        self.stamina_depleted_cooldown = 0.0 # Contador para o cooldown da estamina

        # Superfície temporária para desenho com transparência do agente
        self.agent_surface = pygame.Surface((AGENT_SIZE * 2, AGENT_SIZE * 2), pygame.SRCALPHA)
        # Superfície temporária para desenho com transparência do FOV
        self.fov_surface = pygame.Surface((VISION_RANGE * 2, VISION_RANGE * 2), pygame.SRCALPHA)


    def get_inputs(self, closest_fruit: Optional[Fruit]) -> np.ndarray:
        # Se não há fruta, retorna inputs indicando que não há fruta próxima
        if closest_fruit is None:
            # Ângulo 0, Distância máxima (1.0), Fruta_a_frente 0
            return np.array([0.0, 1.0, 0.0]) 

        vec_to_fruit = closest_fruit.pos - self.pos
        distance = vec_to_fruit.length()
        
        if distance < 1e-5: # Evitar divisão por zero se estiver sobre a fruta
             # Ângulo 0, Distância 0, Fruta_a_frente 1 (muito perto)
             return np.array([0.0, 0.0, 1.0]) 

        # 1. Ângulo da fruta em relação à direção do agente
        angle_to_fruit = math.atan2(vec_to_fruit.y, vec_to_fruit.x)
        relative_angle = angle_to_fruit - self.angle
        # Normalizar ângulo para o intervalo [-pi, pi]
        relative_angle = (relative_angle + math.pi) % (2 * math.pi) - math.pi
        
        # 2. Distância até a fruta (normalizada pela visão)
        normalized_distance = min(1.0, distance / VISION_RANGE)

        # 3. Tem fruta na frente? (dentro do campo de visão e alcance)
        fruit_in_front = 1.0 if (abs(relative_angle) < FIELD_OF_VIEW_ANGLE / 2 and distance < VISION_RANGE) else 0.0

        # Normalizar ângulo relativo para o intervalo [-1, 1]
        normalized_relative_angle = relative_angle / math.pi
        
        return np.array([normalized_relative_angle, normalized_distance, fruit_in_front])

    def update(self, dt: float, action: np.ndarray):
        if not self.is_alive:
            return

        self.lifespan += dt
        
        # Ações baseadas na saída da rede neural [-1, 1]
        rotation_action = action[0]  # Saída 1: Rotação
        move_action = action[1]      # Saída 2: Movimento

        # Girar
        self.angle += rotation_action * self.rotation_speed * dt
        self.angle %= (2 * math.pi) # Manter o ângulo entre 0 e 2*pi

        # --- Lógica de Estamina e Velocidade ---
        current_speed_multiplier = 1.0 # Padrão: velocidade normal
        did_move = False # Flag para saber se o agente tentou mover

        if self.use_stamina:
            if self.is_stamina_depleted:
                # Se a estamina está esgotada, o agente está no cooldown e lento
                self.stamina_depleted_cooldown -= dt
                current_speed_multiplier = STAMINA_DEPLETION_SLOW_FACTOR
                
                if self.stamina_depleted_cooldown <= 0:
                    # Cooldown terminou, resetar o estado de estamina
                    self.is_stamina_depleted = False
                    self.stamina = MAX_STAMINA # Estamina totalmente recarregada
                    self.stamina_depleted_cooldown = 0.0
                    current_speed_multiplier = 1.0 # Volta à velocidade normal
            else:
                # Agente não está no estado de estamina esgotada, gerenciar estamina normalmente
                if move_action > 0.1: # Agente está tentando se mover
                    if self.stamina > 0:
                        self.stamina -= STAMINA_COST_MOVE * dt
                        if self.stamina < 0: self.stamina = 0
                    else:
                        # Estamina acabou, entrar no estado de esgotamento
                        self.is_stamina_depleted = True
                        self.stamina_depleted_cooldown = STAMINA_DEPLETION_COOLDOWN_DURATION
                        current_speed_multiplier = STAMINA_DEPLETION_SLOW_FACTOR # Já fica lento imediatamente
                else: # Agente não está tentando se mover, regenera estamina
                    self.stamina = min(MAX_STAMINA, self.stamina + (STAMINA_COST_MOVE / 2) * dt)
        
        # --- Lógica de Movimento ---
        if move_action > 0.1: # Agente tenta se mover
            did_move = True
            direction = pygame.math.Vector2(math.cos(self.angle), math.sin(self.angle))
            # Aplica o multiplicador de velocidade ao movimento
            self.pos += direction * self.base_speed * current_speed_multiplier * dt * move_action
        
        # Manter o agente dentro dos limites do mundo (wrap around)
        self.pos.x %= self.world_size[0]
        self.pos.y %= self.world_size[1]
        
        # Perder vida ao longo do tempo
        self.life -= LIFE_DECAY_RATE * dt * 5 # Fator 5 para acelerar a simulação
        if self.life <= 0:
            self.is_alive = False
            self.life = 0 # Garante que a vida não fique negativa

    def eat(self):
        self.life = min(MAX_LIFE, self.life + LIFE_GAIN_ON_EAT)

    def draw(self, surface: pygame.Surface, is_semi_transparent: bool, draw_fov: bool):
        current_color = AGENT_ACTIVE_COLOR if self.is_alive else AGENT_DEAD_COLOR

        # Offset para centralizar o desenho local na posição do agente.
        agent_local_center_offset = pygame.math.Vector2(AGENT_SIZE, AGENT_SIZE)
        
        if is_semi_transparent:
            # --- Desenhar FOV (cone de visão) ---
            if draw_fov and self.is_alive: # Desenha FOV apenas se o agente estiver vivo
                self.fov_surface.fill((0, 0, 0, 0)) # Limpa com cor transparente
                
                # Centro da superfície FOV local para o triângulo do FOV
                fov_local_origin = pygame.math.Vector2(VISION_RANGE, VISION_RANGE)
                
                # Pontos do cone do FOV em relação ao fov_local_origin
                p_fov_origin = fov_local_origin # Ponta do cone no centro do agente
                p_fov_left = pygame.math.Vector2(VISION_RANGE, 0).rotate_rad(self.angle - FIELD_OF_VIEW_ANGLE / 2) + fov_local_origin
                p_fov_right = pygame.math.Vector2(VISION_RANGE, 0).rotate_rad(self.angle + FIELD_OF_VIEW_ANGLE / 2) + fov_local_origin
                
                pygame.draw.polygon(self.fov_surface, FOV_COLOR + (FOV_ALPHA,), [p_fov_origin, p_fov_left, p_fov_right])
                # Blit a superfície FOV, deslocada para que seu centro coincida com a posição do agente
                surface.blit(self.fov_surface, self.pos - fov_local_origin)

            # --- Desenhar Agente (triângulo) ---
            self.agent_surface.fill((0, 0, 0, 0)) # Limpa com cor transparente
            
            # Pontos do triângulo relativo ao centro da agent_surface
            local_p1 = pygame.math.Vector2(AGENT_SIZE, 0).rotate_rad(self.angle) + agent_local_center_offset
            local_p2 = pygame.math.Vector2(-AGENT_SIZE/2, -AGENT_SIZE/2).rotate_rad(self.angle) + agent_local_center_offset
            local_p3 = pygame.math.Vector2(-AGENT_SIZE/2, AGENT_SIZE/2).rotate_rad(self.angle) + agent_local_center_offset

            pygame.draw.polygon(self.agent_surface, current_color + (100,), [local_p1, local_p2, local_p3])
            # Blit a superfície do agente, deslocada para que seu centro coincida com a posição do agente
            surface.blit(self.agent_surface, self.pos - agent_local_center_offset)
        else:
            # Desenha o agente diretamente na tela principal (opaco)
            p1 = pygame.math.Vector2(AGENT_SIZE, 0).rotate_rad(self.angle) + self.pos
            p2 = pygame.math.Vector2(-AGENT_SIZE/2, -AGENT_SIZE/2).rotate_rad(self.angle) + self.pos
            p3 = pygame.math.Vector2(-AGENT_SIZE/2, AGENT_SIZE/2).rotate_rad(self.angle) + self.pos
            pygame.draw.polygon(surface, current_color, [p1, p2, p3])

        # --- Renderizar barras de vida e estamina (sempre opacas) ---
        bar_width = AGENT_SIZE * 2
        bar_height = 3
        bar_offset_y = AGENT_SIZE + 2 # Espaço abaixo do agente

        # Posição base para as barras (centralizada abaixo do agente)
        bar_base_x = self.pos.x - AGENT_SIZE
        bar_base_y = self.pos.y + bar_offset_y

        # Barra de Vida
        life_percentage = self.life / MAX_LIFE
        life_bar_color = (0, 200, 0) if self.is_alive else (50, 50, 50) # Verde se vivo, cinza escuro se morto
        
        pygame.draw.rect(surface, (50, 50, 50), (bar_base_x, bar_base_y, bar_width, bar_height)) # Fundo da barra
        pygame.draw.rect(surface, life_bar_color, (bar_base_x, bar_base_y, bar_width * life_percentage, bar_height)) # Preenchimento

        # Barra de Estamina (se ativada)
        if self.use_stamina:
            stamina_percentage = self.stamina / MAX_STAMINA
            stamina_bar_color = (0, 0, 200) if not self.is_stamina_depleted else (100, 100, 255) # Azul normal ou um azul mais claro/indicador de cooldown
            
            bar_base_y += bar_height + 2 # Posição abaixo da barra de vida
            pygame.draw.rect(surface, (50, 50, 50), (bar_base_x, bar_base_y, bar_width, bar_height)) # Fundo da barra
            pygame.draw.rect(surface, stamina_bar_color, (bar_base_x, bar_base_y, bar_width * stamina_percentage, bar_height)) # Preenchimento


class GameWorld:
    def __init__(self, nn: NN, world_size: Tuple[int, int], num_fruits: int, use_stamina: bool):
        self.nn = nn # Guarda a NN original para o retorno (flattened_parameters)
        self.world_size = world_size
        self.agent = Agent(nn, world_size, use_stamina)
        self.fruits = [Fruit(world_size) for _ in range(num_fruits)]
        self._lifespan_recorded = False

    def find_closest_fruit(self) -> Optional[Fruit]:
        if not self.agent.is_alive:
            return None # Agente morto não precisa de fruta

        closest_fruit = None
        min_dist_sq = float('inf')
        for fruit in self.fruits:
            dist_sq = self.agent.pos.distance_squared_to(fruit.pos)
            if dist_sq < min_dist_sq:
                min_dist_sq = dist_sq
                closest_fruit = fruit
        return closest_fruit

    def update(self, dt: float):
        if not self.agent.is_alive:
            return # Não atualiza agentes mortos

        closest_fruit = self.find_closest_fruit()
        
        # Tomada de decisão: Passa a fruta mais próxima (ou None) para obter inputs
        inputs = self.agent.get_inputs(closest_fruit)
        actions = self.agent.nn.predict(inputs)[0] # A predição retorna um array 2D

        # Atualização do estado
        self.agent.update(dt, actions)
        
        # Checar colisão com frutas
        if self.agent.is_alive and closest_fruit is not None:
            if self.agent.pos.distance_to(closest_fruit.pos) < AGENT_SIZE: # Colisão com a fruta mais próxima
                self.agent.eat()
                closest_fruit.respawn()
    
    def draw(self, surface: pygame.Surface, is_semi_transparent: bool, draw_fov: bool):
        # Frutas são desenhadas com transparência se o modo 'is_semi_transparent' estiver ativo
        for fruit in self.fruits:
            fruit.draw(surface, is_semi_transparent=is_semi_transparent)
        
        # Agente (e suas barras) são desenhados com transparência e FOV se solicitado
        self.agent.draw(surface, is_semi_transparent, draw_fov)

# ==============================================================================
# 3. FUNÇÃO PRINCIPAL DA SIMULAÇÃO
# ==============================================================================

def run_game_simulation(
    neural_networks: List[NN],
    world_size: Tuple[int, int] = (800, 600),
    num_fruits: int = 10,
    use_stamina: bool = False,
    render_percentage: float = 0.1,
    skip_frames: int = 0,
    render_fov: bool = False # Nova opção para renderizar o FOV
) -> List[Tuple[np.ndarray, float]]:
    """
    Roda a simulação do jogo com múltiplos agentes. Implementa um loop de jogo
    com fixed timestep para garantir que a lógica da simulação avance de forma
    consistente, desacoplada da taxa de renderização.

    Args:
        neural_networks (List[NN]): Uma lista de redes neurais, uma para cada agente.
        world_size (Tuple[int, int]): O tamanho (largura, altura) do mundo do jogo.
        num_fruits (int): Quantidade de frutas em cada mapa.
        use_stamina (bool): Se os agentes devem ter e usar estamina.
        render_percentage (float): A porcentagem de agentes a serem renderizados (0.0 a 1.0).
        skip_frames (int): Quantidade de quadros a pular entre renderizações.
                           -1 para não renderizar nada (modo headless).
                           0 para renderizar todos os frames.
                           N > 0 para pular N frames (renderiza 1 a cada N+1 frames).
        render_fov (bool): Se o campo de visão do agente deve ser renderizado.

    Returns:
        List[Tuple[np.ndarray, float]]: Uma lista de tuplas contendo
                                        (parâmetros achatados da NN do agente, tempo de vida).
    """
    if skip_frames != -1:
        pygame.init()
        screen = pygame.display.set_mode(world_size)
        pygame.display.set_caption("Simulação de Agentes Inteligentes")
        font = pygame.font.SysFont(None, 24)
    
    clock = pygame.time.Clock()

    print(f"Iniciando simulação com {len(neural_networks)} agentes.")
    
    # Constantes para o fixed timestep
    FIXED_FPS = 60 # A simulação lógica sempre avança como se estivesse a 60 FPS
    FIXED_DT = 1.0 / FIXED_FPS # O passo de tempo fixo para cada atualização do agente

    # Cria um mundo para cada rede neural
    worlds = [GameWorld(nn, world_size, num_fruits, use_stamina) for nn in neural_networks]
    
    # Seleciona quais agentes serão renderizados (primeira seleção)
    num_to_render = int(len(worlds) * render_percentage)
    indices_to_render = random.sample(range(len(worlds)), k=min(len(worlds), num_to_render))
    print(f"Renderizando {len(indices_to_render)} agentes.")
    
    # Lista para armazenar (parâmetros da NN, tempo de vida)
    agent_results = []
    
    running = True
    
    accumulator = 0.0 # Acumula o tempo real passado, para ser consumido em passos de simulação fixos
    logical_simulation_frame_counter = 0 # Conta quantos passos de simulação fixos (frames lógicos) ocorreram
    
    # Inicializa o tempo real para calcular o real_dt
    last_real_time_ms = pygame.time.get_ticks() if skip_frames != -1 else int(time.time() * 1000)
    start_time = time.time()

    while running:
        # 1. Calcula o tempo real decorrido desde a última iteração do loop principal.
        # Usa pygame.time.get_ticks() se a renderização estiver ativa, ou time.time() para modo headless.
        current_real_time_ms = pygame.time.get_ticks() if skip_frames != -1 else int(time.time() * 1000)
        real_dt = (current_real_time_ms - last_real_time_ms) / 1000.0
        last_real_time_ms = current_real_time_ms

        # Limita o real_dt para evitar grandes saltos de tempo após pausas ou lags
        if real_dt > 0.1: # Máximo de 100ms por passo de tempo real
            real_dt = 0.1
        accumulator += real_dt

        # 2. Manipulação de eventos Pygame (apenas se a renderização estiver ativa)
        if skip_frames != -1:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
                if event.type == pygame.KEYDOWN: # Garante que só reage a teclas pressionadas
                    if event.key == pygame.K_SPACE: # Alternar agentes renderizados
                        alive_world_indices = [i for i, world in enumerate(worlds) if world.agent.is_alive]
                        if len(alive_world_indices) > 0:
                            num_to_sample = min(len(alive_world_indices), num_to_render)
                            indices_to_render = random.sample(alive_world_indices, k=num_to_sample)
                            print(f"Alternando para renderizar {len(indices_to_render)} novos agentes.")
                        else:
                            indices_to_render = []
                            print("Nenhum agente vivo para renderizar.")
                    if event.key == pygame.K_f: # Alternar renderização do FOV
                        render_fov = not render_fov
                        print(f"Renderização do FOV: {'Ativada' if render_fov else 'Desativada'}")

        # 3. Atualizações da simulação (com passo de tempo fixo)
        # Consome o tempo acumulado em passos de simulação fixos (FIXED_DT).
        # A lógica da simulação sempre avança em incrementos de FIXED_DT.
        while accumulator >= FIXED_DT and running: # `and running` para sair se um evento QUIT ocorreu
            for i, world in enumerate(worlds):
                if world.agent.is_alive:
                    world.update(FIXED_DT) # Todos os agentes são atualizados com o FIXED_DT
                elif not world._lifespan_recorded: # Se o agente morreu e seu tempo de vida não foi registrado
                    agent_results.append((flatten_parameters(world.nn), world.agent.lifespan))
                    world._lifespan_recorded = True
            
            accumulator -= FIXED_DT
            logical_simulation_frame_counter += 1

            # Verifica se todos os agentes morreram dentro deste passo de simulação
            active_agents = sum(1 for world in worlds if world.agent.is_alive)
            if active_agents == 0:
                print("Todos os agentes morreram. Fim da simulação.")
                running = False
                break # Sai do loop `while accumulator`

        if not running: # Se a simulação terminou, sai também do loop principal
            break

        # 4. Renderização (se habilitada e for um frame de renderização designado)
        # `skip_frames == -1` significa nenhuma renderização.
        should_render_this_pass = False
        if skip_frames != -1: # Verifica se a renderização não está desabilitada
            # Renderiza se o contador de frames lógicos for maior que 0
            # E se o frame lógico atual se alinha com o intervalo de renderização (skip_frames + 1)
            if logical_simulation_frame_counter > 0 and (logical_simulation_frame_counter % (skip_frames + 1) == 0):
                should_render_this_pass = True

        if should_render_this_pass:
            screen.fill((10, 20, 40)) # Fundo azul escuro
            
            # Desenha apenas os agentes selecionados
            for i in indices_to_render:
                worlds[i].draw(screen, is_semi_transparent=True, draw_fov=render_fov)
            
            # Atualiza a contagem de agentes ativos para exibição
            # (já foi calculada no final do loop `while accumulator`)
            
            # Mostra informações
            # clock.get_fps() agora reflete a taxa de frames *renderizados*
            info_text = (f"Agentes Vivos: {active_agents}/{len(worlds)} | FPS: {clock.get_fps():.1f} | "
                         f"FOV: {'ON' if render_fov else 'OFF'} | Sim Frames: {logical_simulation_frame_counter}")
            text_surface = font.render(info_text, True, (255, 255, 255))
            screen.blit(text_surface, (10, 10))
            
            pygame.display.flip()
            
            # Isso limita a taxa de frames *renderizados* ao FIXED_FPS.
            # É importante que isso ocorra APÓS a renderização.
    
    # Finalização
    end_time = time.time()
    print(f"Simulação concluída em {end_time - start_time:.2f} segundos.")

    if skip_frames != -1:
        pygame.quit()
        
    # Adiciona os resultados dos agentes que ainda estavam vivos no final
    for world in worlds:
        if not world._lifespan_recorded:
            agent_results.append((flatten_parameters(world.nn), world.agent.lifespan))

    return agent_results

# ==============================================================================
# 4. BLOCO DE EXEMPLO DE USO
# ==============================================================================

if __name__ == '__main__':
    # --- Parâmetros da Simulação ---
    NUM_AGENTS = 100 # Um número maior para ver a alternância
    USE_STAMINA_OPTION = True
    RENDER_PERCENT = 0.1  # Mostrar 10% dos agentes (10 agentes neste caso)
    SKIP_FRAMES_OPTION = 0 # 0 = renderizar todos os frames, 5 = pular 5 frames, -1 = não renderizar (headless)
    RENDER_FOV_OPTION = True # Renderizar o campo de visão por padrão

    # --- Criação das Redes Neurais ---
    # Cada agente precisa de 3 entradas e 2 saídas
    # Entradas: angulo_relativo, distancia, fruta_a_frente
    # Saídas: girar, andar (valores entre -1 e 1)
    input_neurons = 3
    hidden_neurons = 6
    output_neurons = 2
    
    agent_networks = [
        NN(inputs=input_neurons, hidden_units=hidden_neurons, outputs=output_neurons, activation='tanh')
        for _ in range(NUM_AGENTS)
    ]

    # --- Executar a Simulação ---
    results = run_game_simulation(
        neural_networks=agent_networks,
        use_stamina=USE_STAMINA_OPTION,
        render_percentage=RENDER_PERCENT,
        skip_frames=SKIP_FRAMES_OPTION,
        num_fruits=15,
        render_fov=RENDER_FOV_OPTION
    )

    # --- Mostrar Resultados ---
    if results:
        # Extrai apenas os tempos de vida para estatísticas
        lifespans = [r[1] for r in results] 
        print("\n--- Resultados da Simulação ---")
        print(f"Total de agentes: {len(lifespans)}")
        print(f"Tempo de vida médio: {np.mean(lifespans):.2f} segundos")
        print(f"Melhor tempo de vida: {np.max(lifespans):.2f} segundos")
        print(f"Pior tempo de vida: {np.min(lifespans):.2f} segundos")

        # Exemplo de como acessar os parâmetros de uma NN específica (ex: a melhor)
        best_agent_idx = np.argmax(lifespans)
        best_nn_params = results[best_agent_idx][0]
        print(f"\nParâmetros da NN do agente com melhor tempo de vida (top 5 elementos): {best_nn_params[:5]}...")

Iniciando simulação com 100 agentes.
Renderizando 10 agentes.
Simulação concluída em 1.68 segundos.

--- Resultados da Simulação ---
Total de agentes: 100
Tempo de vida médio: 1.67 segundos
Melhor tempo de vida: 1.67 segundos
Pior tempo de vida: 1.67 segundos

Parâmetros da NN do agente com melhor tempo de vida (top 5 elementos): [-1.26467426 -0.09094305  0.38593512  0.56503204  0.63857716]...


In [ ]:
import time
import numpy as np
from random import randint
from typing import List

INPUTS = 1
HIDDEN_UNITS = 10
OUTPUT = 1
ACTIVATION = 'relu'


class GeneticOptimizationAgent:
    def __init__(self, probability_mutation=0.01, keep_better=False, tamanho_genoma=10, tamanho_pop=1000):
        self.tamanho_genoma = tamanho_genoma
        self.tamanho_pop = tamanho_pop
        self.probability_mutation = probability_mutation

        self.population = [self.gerar_individuo_aleatoriamente() for i in range(self.tamanho_pop)]

        self.keep_better = keep_better

    @staticmethod    
    def gerar_individuo_aleatoriamente_estatico(tamanho_genoma):
        return np.random.normal(0, 1, tamanho_genoma)


    def gerar_individuo_aleatoriamente(self):
        return np.random.normal(0, 1, self.tamanho_genoma)

    
    def fitness(self):
        return run_game_simulation(
            neural_networks=self.population,
            use_stamina=USE_STAMINA_OPTION,
            render_percentage=RENDER_PERCENT,
            skip_frames=SKIP_FRAMES_OPTION,
            num_fruits=5,
            render_fov=RENDER_FOV_OPTION
        )

    def mutate(self, individual: np.array) -> np.array:
        if len(individual) != self.tamanho_genoma:
            raise Exception("Indivíduo de tamanho errado")
        
        mutated = individual.copy()
        for i in range(len(mutated)):
            if np.random.rand() < self.probability_mutation:
                mutated[i] += np.random.normal(0, 1)

        return mutated
    
    @staticmethod
    def crossover(ind1: np.array, ind2: np.array) -> List[np.array]:
        if len(ind1) != len(ind2):
            raise Exception("Indivíduos de tamanhos diferentes")
        
        tamanho_genoma = len(ind1)
        
        cross_point = np.random.randint(1, tamanho_genoma - 1)

        # Use np.concatenate to join numpy array slices
        child1 = np.concatenate((ind1[:cross_point], ind2[cross_point:]))
        child2 = np.concatenate((ind2[:cross_point], ind1[cross_point:]))

        return [child1, child2]
    
    def selection(self):
        # Implementando torneiro

        new_population = []

        MSE = 0

        fitness_tuple = self.fitness()

        MSE = -max([x[1] for x in fitness_tuple])

        for i in range(self.tamanho_pop//2):
            ind1_index = randint(0, self.tamanho_pop - 1)
            ind2_index = randint(0, self.tamanho_pop - 1)

            pai = fitness_tuple[ind1_index][0] if fitness_tuple[ind1_index][1] > fitness_tuple[ind2_index][1] else fitness_tuple[ind2_index][0]

            ind1_index = randint(0, self.tamanho_pop - 1)
            ind2_index = randint(0, self.tamanho_pop - 1)

            mae = fitness_tuple[ind1_index][0] if fitness_tuple[ind1_index][1] > fitness_tuple[ind2_index][1] else fitness_tuple[ind2_index][0]

            children = self.crossover(pai, mae)

            # Mutação
            children = [self.mutate(child) for child in children]
            new_population.extend(children)

        if self.keep_better:
            best_individual = max(fitness_tuple, key=lambda x: x[1])[0]
            new_population[0] = best_individual
        

        self.population = new_population

        return best_individual, MSE


#### Testando crossover

In [7]:
"""
ind1 = gerar_individuo_aleatoriamente()
ind2 = gerar_individuo_aleatoriamente()

print(f'Pai: \t\t{ind1}, valor: {genoma_para_valor(ind1)}')
print(f'Mãe: \t\t{ind2}, valor: {genoma_para_valor(ind2)}')

child1, child2 = GeneticOptimization.crossover(ind1, ind2)

print(f'Filho1: \t{child1}, valor: {genoma_para_valor(child1)}')
print(f'Filho2: \t{child2}, valor: {genoma_para_valor(child2)}')
"""

"\nind1 = gerar_individuo_aleatoriamente()\nind2 = gerar_individuo_aleatoriamente()\n\nprint(f'Pai: \t\t{ind1}, valor: {genoma_para_valor(ind1)}')\nprint(f'Mãe: \t\t{ind2}, valor: {genoma_para_valor(ind2)}')\n\nchild1, child2 = GeneticOptimization.crossover(ind1, ind2)\n\nprint(f'Filho1: \t{child1}, valor: {genoma_para_valor(child1)}')\nprint(f'Filho2: \t{child2}, valor: {genoma_para_valor(child2)}')\n"

In [3]:
import time
import numpy as np
from random import randint
from typing import List

INPUTS = 1
HIDDEN_UNITS = 10
OUTPUT = 1


class GeneticOptimizationAgent:
    def __init__(self, probability_mutation=0.01, keep_better=False, input_neurons=3, hidden_neurons=5, activation_neruons=3, tamanho_pop=1000, activation='sigmoid'):
        self.tamanho_genoma = (input_neurons * hidden_neurons) + hidden_neurons + (hidden_neurons * activation_neruons) + activation_neruons
        self.tamanho_pop = tamanho_pop
        self.input_neurons = input_neurons
        self.hidden_neurons = hidden_neurons
        self.activation_neruons = activation_neruons
        self.activation_function = activation

        self.probability_mutation = probability_mutation

        self.population = [self.gerar_individuo_aleatoriamente() for i in range(self.tamanho_pop)]

        self.keep_better = keep_better
    
    def gerar_individuo_aleatoriamente(self):
        return np.random.normal(0, 1, self.tamanho_genoma)

    
    def fitness(self, skip_frames=5):

        nns = [NN(inputs=self.input_neurons, hidden_units=self.hidden_neurons, outputs=self.activation_neruons, activation=self.activation_function) for _ in range(self.tamanho_pop)]
        for i in range(self.tamanho_pop):
            unflatten_parameters(nns[i], self.population[i])

        return run_game_simulation(
            neural_networks=nns,
            use_stamina=USE_STAMINA_OPTION,
            render_percentage=0.01,
            skip_frames=skip_frames,
            num_fruits=15,
            render_fov=RENDER_FOV_OPTION
        )

    def mutate(self, individual: np.array) -> np.array:
        if len(individual) != self.tamanho_genoma:
            raise Exception("Indivíduo de tamanho errado")
        
        mutated = individual.copy()
        for i in range(len(mutated)):
            if np.random.rand() < self.probability_mutation:
                mutated[i] += np.random.normal(0, 1)

        return mutated
    
    @staticmethod
    def crossover(ind1: np.array, ind2: np.array) -> List[np.array]:
        if len(ind1) != len(ind2):
            raise Exception("Indivíduos de tamanhos diferentes")
        
        tamanho_genoma = len(ind1)
        
        cross_point = np.random.randint(1, tamanho_genoma - 1)

        # Use np.concatenate to join numpy array slices
        child1 = np.concatenate((ind1[:cross_point], ind2[cross_point:]))
        child2 = np.concatenate((ind2[:cross_point], ind1[cross_point:]))

        return [child1, child2]
    
    def selection(self, skip_frames=5):
        # Implementando torneiro

        new_population = []

        MSE = 0

        fitness_tuple = self.fitness(skip_frames=skip_frames)

        MSE = max([x[1] for x in fitness_tuple])

        for i in range(self.tamanho_pop//2):
            ind1_index = randint(0, self.tamanho_pop - 1)
            ind2_index = randint(0, self.tamanho_pop - 1)

            pai = fitness_tuple[ind1_index][0] if fitness_tuple[ind1_index][1] > fitness_tuple[ind2_index][1] else fitness_tuple[ind2_index][0]

            ind1_index = randint(0, self.tamanho_pop - 1)
            ind2_index = randint(0, self.tamanho_pop - 1)

            mae = fitness_tuple[ind1_index][0] if fitness_tuple[ind1_index][1] > fitness_tuple[ind2_index][1] else fitness_tuple[ind2_index][0]

            children = self.crossover(pai, mae)

            # Mutação
            children = [self.mutate(child) for child in children]
            new_population.extend(children)

        if self.keep_better:
            best_individual = max(fitness_tuple, key=lambda x: x[1])[0]
            new_population[0] = best_individual
        

        self.population = new_population

        return best_individual, MSE


In [4]:

# Saídas: girar, andar
input_neurons = 3
hidden_neurons = 6
output_neurons = 2

optimizator = GeneticOptimizationAgent(probability_mutation=0.05,keep_better=True, input_neurons=input_neurons, hidden_neurons=hidden_neurons, activation_neruons=output_neurons, tamanho_pop=1000, activation='tanh')


In [5]:
import matplotlib.pyplot as plt 

epochs = 1000

MSE = np.zeros((epochs))  # Lista para armazenar o MSE de cada é

for generation in range(epochs):
    start_time = time.time()
    best_individual, CMSE = optimizator.selection(skip_frames=-1)
    print(f"Geração {generation}: Melhor indivíduo: {best_individual}, Fitness: {CMSE:.6f}, Tempo: {time.time() - start_time:.2f}s")
    MSE[generation] = CMSE 
    

plt.plot(range(epochs), MSE)
plt.xlabel('Geração')
plt.ylabel('MSE do Melhor Indivíduo')
plt.title('Evolução do MSE ao Longo das Gerações')

Iniciando simulação com 1000 agentes.
Renderizando 10 agentes.
Todos os agentes morreram. Fim da simulação.
Simulação concluída em 24.67 segundos.
Geração 0: Melhor indivíduo: [-1.10346775  0.12384963  1.24591866  1.19517243  0.74689962  1.27619026
  2.06590074  1.56999403 -0.38792873  0.63549059 -0.23465677  0.33990362
  0.48903673 -1.70576346  0.77893712  1.40972723  0.7338087  -1.55605991
  1.07050214  0.21527716 -0.39014812  0.8809771   0.42566747  0.41168476
  0.6150092  -0.66691967  1.24341264  1.14759564  0.74239052  0.40657489
 -1.8220598   0.14251434  0.73335914  1.47697702 -0.16739116  0.88940281
 -1.44806047  0.66735224], Fitness: 24.533333, Tempo: 25.05s
Iniciando simulação com 1000 agentes.
Renderizando 10 agentes.
Todos os agentes morreram. Fim da simulação.
Simulação concluída em 26.44 segundos.
Geração 1: Melhor indivíduo: [ 1.33817177  0.24314429  0.64937209  0.42942908 -0.73807191  0.5827758
  0.56683467  0.56087314 -1.0677687  -0.4415572   1.37234573  0.36615694
 -0.

KeyboardInterrupt: 

In [6]:
optimizator.fitness(skip_frames=0)



Iniciando simulação com 1000 agentes.
Renderizando 10 agentes.
Alternando para renderizar 10 novos agentes.
Alternando para renderizar 10 novos agentes.
Alternando para renderizar 10 novos agentes.
Alternando para renderizar 2 novos agentes.
Alternando para renderizar 1 novos agentes.
Simulação concluída em 29.11 segundos.


[(array([ 0.00726828, -1.96519712,  1.28718097, -0.06885544,  0.41839098,
         -1.25599958,  0.69965682,  1.18639125, -1.4946148 , -0.41857802,
          1.54972442, -0.10041898, -0.33121297,  0.18486109, -1.14831527,
          2.21137065,  1.92637733,  1.13320666, -0.56279352,  1.00692036,
          0.57805601,  0.18782379,  0.26578581,  2.22560393, -0.75037615,
         -0.42080813,  0.63625102, -1.88365755, -0.21325002, -0.31226034,
         -0.41036903,  2.00070121, -0.45617935,  0.45687045, -0.16119556,
          1.14012304, -1.80559308,  0.51528217]),
  6.666666666666648),
 (array([ 9.91426499e-01, -7.06588711e-01,  3.25205607e-01,  2.44478212e+00,
         -1.62364173e-01, -1.66559101e-01,  1.83925372e+00, -1.93387551e+00,
          1.44978869e+00, -1.27336106e+00, -1.43178652e+00, -1.08867505e+00,
          2.27083983e+00, -1.00119374e+00, -3.88542653e-01, -6.19434781e-01,
          1.33365090e+00,  2.13790117e-02,  4.77417594e-01,  1.43122855e+00,
          1.28118784e+00,

# Jogo Complexo: código

### Gerador procedural de labirinto

In [7]:
import random

def gerar_labirinto(largura, altura):
    """
    Gera um labirinto proceduralmente usando o algoritmo Recursive Backtracking.
    
    O labirinto é "perfeito", o que significa que é completamente ligado e 
    sem ciclos (existe sempre um único caminho entre dois pontos quaisquer).

    Args:
        largura (int): A largura do labirinto em número de células.
        altura (int): A altura do labirinto em número de células.

    Returns:
        list[list[int]]: Uma matriz representando o labirinto, onde 1 é parede e 0 é caminho.
    """
    if largura <= 0 or altura <= 0:
        raise ValueError("A largura e a altura devem ser maiores que zero.")

    # O tamanho real da matriz precisa ser 2*n + 1 para acomodar as paredes entre as células.
    largura_matriz = largura * 2 + 1
    altura_matriz = altura * 2 + 1
    
    # Inicia o labirinto com todas as posições como parede (1)
    labirinto = [[1 for _ in range(largura_matriz)] for _ in range(altura_matriz)]
    
    # Pilha para o algoritmo de busca em profundidade (DFS)
    pilha = []
    
    # Escolhe uma célula inicial aleatória (coordenadas sempre ímpares na matriz)
    # Células: (1,1), (1,3), (3,1), etc.
    # Paredes: (0,x), (x,0), (2,x), (x,2), etc.
    cx, cy = (random.randint(0, largura - 1) * 2 + 1, 
              random.randint(0, altura - 1) * 2 + 1)
              
    labirinto[cy][cx] = 0  # Marca a célula inicial como caminho
    pilha.append((cx, cy))
    
    while pilha:
        cx, cy = pilha[-1] # Pega a célula atual do topo da pilha
        
        # Encontra todos os vizinhos não visitados (a 2 passos de distância)
        vizinhos = []
        # Norte
        if cy - 2 >= 0 and labirinto[cy - 2][cx] == 1:
            vizinhos.append((cx, cy - 2))
        # Sul
        if cy + 2 < altura_matriz and labirinto[cy + 2][cx] == 1:
            vizinhos.append((cx, cy + 2))
        # Leste
        if cx + 2 < largura_matriz and labirinto[cy][cx + 2] == 1:
            vizinhos.append((cx + 2, cy))
        # Oeste
        if cx - 2 >= 0 and labirinto[cy][cx - 2] == 1:
            vizinhos.append((cx - 2, cy))
            
        if vizinhos:
            # Escolhe um vizinho aleatório
            nx, ny = random.choice(vizinhos)
            
            # Derruba a parede entre a célula atual e o vizinho
            parede_x, parede_y = (cx + nx) // 2, (cy + ny) // 2
            labirinto[parede_y][parede_x] = 0
            
            # Marca o vizinho como caminho e o adiciona à pilha
            labirinto[ny][nx] = 0
            pilha.append((nx, ny))
        else:
            # Se não há vizinhos não visitados, retrocede (backtrack)
            pilha.pop()
            
    # Cria uma entrada e uma saída
    labirinto[1][0] = 0  # Entrada no canto superior esquerdo
    labirinto[altura_matriz - 2][largura_matriz - 1] = 0 # Saída no canto inferior direito

    return labirinto

def gerar_labirinto_quadrado(tamanho):
    return gerar_labirinto(tamanho, tamanho)

# --- Exemplo de Uso ---
if __name__ == "__main__":
    # Gera um labirinto com 20 células de largura e 15 de altura
    # A matriz resultante terá tamanho (15*2+1) x (20*2+1) = 31x41
    largura_celulas = 20
    altura_celulas = 15
    
    meu_labirinto = gerar_labirinto(largura_celulas, altura_celulas)
    
    # Imprime o labirinto de forma visual
    print(f"Labirinto gerado com {altura_celulas}x{largura_celulas} células (matriz {len(meu_labirinto)}x{len(meu_labirinto[0])}):\n")
    for linha in meu_labirinto:
        # Usando caracteres para uma visualização melhor (opcional)
        # █ para parede, ' ' para caminho
        print("".join(['██' if celula == 1 else '  ' for celula in linha]))

    # Se você precisar da matriz de números, basta imprimir diretamente:
    # print(meu_labirinto)

Labirinto gerado com 15x20 células (matriz 31x41):

██████████████████████████████████████████████████████████████████████████████████
            ██                      ██              ██          ██              ██
██  ██████████  ██████  ██████  ██████  ██████████  ██  ██  ██████  ██  ██████████
██  ██      ██  ██      ██  ██  ██      ██      ██      ██          ██          ██
██  ██  ██  ██  ██  ██████  ██  ██  ██████  ██  ██████████  ██████████████████  ██
██      ██  ██  ██  ██      ██  ██          ██  ██      ██  ██              ██  ██
██  ██████  ██  ██  ██████  ██  ██████████████  ██  ██  ██████  ██████████  ██  ██
██  ██      ██  ██      ██              ██      ██  ██      ██  ██              ██
██  ██  ██████  ██████  ██████████████████  ██████  ██████  ██  ██████████████  ██
██  ██  ██      ██  ██      ██              ██  ██  ██          ██          ██  ██
██  ██  ██  ██████  ██████  ██  ██████████████  ██  ██████████████  ██████  ██████
██  ██      ██          ██      ██ 

### Jogo

In [14]:
import pygame
import numpy as np
import math
import os
import random
from typing import List, Callable, Optional, Tuple, Any

# ==============================================================================
# 1. CÓDIGO DA REDE NEURAL (FORNECIDO E ADAPTADO PARA ESTE CONTEXTO)
# ==============================================================================

class NN:
    """
    Uma classe para representar uma rede neural simples com uma camada oculta.
    """
    def __init__(self, inputs: int, hidden_units: int, outputs: int, activation: str = 'tanh'):
        """
        Inicializa a rede neural com pesos e biases aleatórios.
        Os pesos são inicializados com He/Xavier para melhor desempenho.
        """
        self.inputs = inputs
        self.hidden_units = hidden_units
        self.outputs = outputs
        self.activation_str = activation
        
        # Ordem das dimensões para dot product (inputs, hidden_units) e (hidden_units, outputs)
        self.W1 = np.random.randn(inputs, hidden_units).astype(np.float32) * np.sqrt(2. / inputs)
        self.b1 = np.zeros((1, hidden_units), dtype=np.float32)
        self.W2 = np.random.randn(hidden_units, outputs).astype(np.float32) * np.sqrt(2. / hidden_units)
        self.b2 = np.zeros((1, outputs), dtype=np.float32)
        
        self.actv = self._get_activation_function(activation)

    def _get_activation_function(self, name: str) -> Callable[[np.ndarray], np.ndarray]:
        """Retorna a função de ativação correspondente ao nome."""
        match name:
            case 'sigmoid': return lambda x: 1 / (1 + np.exp(-x))
            case 'tanh': return np.tanh
            case 'relu': return lambda x: np.maximum(0, x)
            case _: raise ValueError(f"Função de ativação '{name}' não suportada.")

    def predict(self, X: np.ndarray) -> np.ndarray:
        """
        Realiza a passagem para a frente (forward pass) na rede para fazer uma predição.
        X deve ser um array 2D (batch_size, input_features).
        """
        if X.ndim == 1: X = X.reshape(1, -1)
        
        # Camada oculta
        # Note: A matriz de pesos W1 é (inputs, hidden_units)
        # X é (1, inputs)
        # X @ W1 é (1, hidden_units)
        z1 = X @ self.W1 + self.b1
        a1 = self.actv(z1)
        
        # Camada de saída
        # a1 é (1, hidden_units)
        # W2 é (hidden_units, outputs)
        # a1 @ W2 é (1, outputs)
        z2 = a1 @ self.W2 + self.b2
        
        # Usar tanh na saída para manter as ações entre -1 e 1
        output = np.tanh(z2) 
        
        return output

    def __repr__(self):
        """Representação em string do objeto NN."""
        return (f"NN(inputs={self.inputs}, hidden_units={self.hidden_units}, "
                f"outputs={self.outputs}, activation='{self.activation_str}')")

# --- Funções para Achatar e Reconstruir os Parâmetros ---

def flatten_parameters(nn: NN) -> np.ndarray:
    """
    Pega uma rede neural (NN) e coloca todos os seus parâmetros (pesos e biases)
    alinhados em um único array numpy. A ordem W1, b1, W2, b2 é importante.
    """
    params = [nn.W1, nn.b1, nn.W2, nn.b2]
    return np.concatenate([p.ravel() for p in params])

def unflatten_parameters(nn: NN, flat_params: np.ndarray):
    """
    Pega um array 1D de parâmetros e os usa para redefinir os pesos e biases
    de uma rede neural (NN) existente.
    """
    s_w1, s_b1, s_w2, s_b2 = nn.W1.size, nn.b1.size, nn.W2.size, nn.b2.size
    total_params = s_w1 + s_b1 + s_w2 + s_b2
    if flat_params.size != total_params:
        raise ValueError(f"Tamanho do array incorreto. Esperado: {total_params}, Recebido: {flat_params.size}")

    pointer = 0
    nn.W1 = flat_params[pointer : pointer + s_w1].reshape(nn.W1.shape)
    pointer += s_w1
    nn.b1 = flat_params[pointer : pointer + s_b1].reshape(nn.b1.shape)
    pointer += s_b1
    nn.W2 = flat_params[pointer : pointer + s_w2].reshape(nn.W2.shape)
    pointer += s_w2
    nn.b2 = flat_params[pointer : pointer + s_b2].reshape(nn.b2.shape)


# ==============================================================================
# 2. CONSTANTES E CONFIGURAÇÕES DO JOGO
# ==============================================================================
# Tela
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 800
CELL_SIZE = 20

# Cores
COLOR_BG = (10, 10, 10)
COLOR_WALL = (100, 100, 100)
COLOR_FLOOR = (30, 30, 30)
COLOR_AI = (255, 50, 50) # Cor base para agentes, será randomizada
COLOR_FOOD = (50, 255, 50)
COLOR_HEALTH_BAR_BG = (50, 50, 50)
COLOR_HEALTH_BAR_FULL = (0, 200, 0)
COLOR_HEALTH_BAR_LOW = (200, 0, 0)
COLOR_DEAD_AGENT = (50, 50, 50, 100) # Cor para agentes que morreram (com transparência)

# Cores para os raios de detecção (com alpha para semi-transparência)
COLOR_RAY_WALL = (255, 255, 0, 80)   # Amarelo
COLOR_RAY_AGENT = (255, 100, 100, 80) # Vermelho claro
COLOR_RAY_FOOD = (100, 255, 100, 80)  # Verde claro

# Parâmetros do Agente
AGENT_SIZE = 15
AGENT_SPEED = 2.0
AGENT_ROTATION_SPEED = 0.05
MAX_RAY_DISTANCE = 200 # Distância máxima de visão dos raios

# Parâmetros de Vida e Comida
MAX_HEALTH = 100
HEALTH_DECAY_RATE = 1.0 # Vida perdida por SEGUNDO (multiplicado por dt no update)
FOOD_HEAL_AMOUNT = 30 # Vida restaurada ao comer
HEALTH_BAR_WIDTH = AGENT_SIZE * 2
HEALTH_BAR_HEIGHT = 5
HEALTH_BAR_OFFSET_Y = AGENT_SIZE + 5 # Distância da barra de vida ao agente

# Parâmetros de Geração
NUM_INITIAL_FOOD = 10

maze = gerar_labirinto_quadrado(20) 

# --- LABIRINTO (Matriz NumPy) ---
open_box_maze = [
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
]
MAZE = np.array(maze, dtype=np.int8)
MAZE_HEIGHT, MAZE_WIDTH = MAZE.shape

# --- Python / NumPy Optimized Functions (substituindo Numba) ---

def _check_collision_python(x: float, y: float, maze: np.ndarray, cell_size: int, maze_width: int, maze_height: int) -> bool:
    """Verifica se uma posição (x,y) está dentro de uma parede."""
    grid_x = int(x / cell_size)
    grid_y = int(y / cell_size)
    if 0 <= grid_x < maze_width and 0 <= grid_y < maze_height:
        return maze[grid_y, grid_x] == 1
    return True # Colide se estiver fora do mapa

def _intersect_ray_circle_python(ray_origin_x: float, ray_origin_y: float, ray_dir_x: float, ray_dir_y: float, circle_center_x: float, circle_center_y: float, radius: float) -> float:
    """Calcula a interseção de um raio com um círculo. Retorna a distância ou -1.0 se não houver interseção válida."""
    oc_x = ray_origin_x - circle_center_x
    oc_y = ray_origin_y - circle_center_y
    
    ray_dir_len_sq = ray_dir_x**2 + ray_dir_y**2
    if ray_dir_len_sq == 0: return -1.0 # Evita divisão por zero
    
    # Normalizar ray_dir_x e ray_dir_y para simplificar 'a' na equação quadrática para 1.0
    # No caso de um vetor unitário, ray_dir_x_norm = ray_dir_x e ray_dir_y_norm = ray_dir_y
    # Mas aqui, estamos usando o vetor de direção não normalizado do ângulo, então precisamos normalizar.
    ray_dir_len = math.sqrt(ray_dir_len_sq)
    ray_dir_x_norm = ray_dir_x / ray_dir_len
    ray_dir_y_norm = ray_dir_y / ray_dir_len

    a = 1.0 # (ray_dir_x_norm**2 + ray_dir_y_norm**2) which is 1.0
    b = 2.0 * (oc_x * ray_dir_x_norm + oc_y * ray_dir_y_norm)
    c = oc_x**2 + oc_y**2 - radius**2
    
    discriminant = b**2 - 4*a*c
    
    if discriminant < 0:
        return -1.0 # Nenhuma interseção real
    
    sqrt_d = math.sqrt(discriminant)
    t1 = (-b - sqrt_d) / 2.0
    t2 = (-b + sqrt_d) / 2.0

    # Queremos a menor distância positiva (interseções à frente da origem do raio)
    if t1 >= 0 and t2 >= 0:
        return min(t1, t2)
    elif t1 >= 0:
        return t1
    elif t2 >= 0:
        return t2
    return -1.0 # Ambas as interseções estão atrás da origem do raio


def _cast_rays_core_python(
    agent_x: float, agent_y: float, agent_angle: float, num_rays: int, delta_theta: float, start_angle_offset: float,
    maze: np.ndarray, cell_size: int, maze_width: int, maze_height: int, max_ray_distance: float,
    other_agent_data: np.ndarray, # Nx4 array: [x, y, size, is_alive (0/1)]
    food_data: np.ndarray         # Mx4 array: [x, y, size, is_eaten (0/1)]
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Lança raios usando DDA para paredes e fórmulas de interseção para objetos (versão Python puro).
    Retorna três arrays NumPy para distâncias de parede, agentes e comida.
    """
    dist_walls = np.full(num_rays, max_ray_distance, dtype=np.float32)
    dist_agents = np.full(num_rays, max_ray_distance, dtype=np.float32)
    dist_food = np.full(num_rays, max_ray_distance, dtype=np.float32)

    for i in range(num_rays): # Use range instead of prange
        angle = agent_angle + start_angle_offset + i * delta_theta
        ray_dir_x, ray_dir_y = math.cos(angle), math.sin(angle)
        
        # --- 1. Detecção de Paredes com DDA ---
        map_x, map_y = int(agent_x / cell_size), int(agent_y / cell_size)
        
        delta_dist_x = abs(1.0 / ray_dir_x) if ray_dir_x != 0 else float('inf')
        delta_dist_y = abs(1.0 / ray_dir_y) if ray_dir_y != 0 else float('inf')
        
        side_dist_x, side_dist_y = 0.0, 0.0
        step_x, step_y = 0, 0

        if ray_dir_x < 0:
            step_x = -1
            side_dist_x = (agent_x % cell_size) * delta_dist_x
        else:
            step_x = 1
            side_dist_x = (cell_size - (agent_x % cell_size)) * delta_dist_x

        if ray_dir_y < 0:
            step_y = -1
            side_dist_y = (agent_y % cell_size) * delta_dist_y
        else:
            step_y = 1
            side_dist_y = (cell_size - (agent_y % cell_size)) * delta_dist_y

        current_dist_wall = 0.0
        
        while current_dist_wall < max_ray_distance:
            if side_dist_x < side_dist_y:
                current_dist_wall = side_dist_x
                side_dist_x += delta_dist_x * cell_size
                map_x += step_x
            else:
                current_dist_wall = side_dist_y
                side_dist_y += delta_dist_y * cell_size
                map_y += step_y

            if not (0 <= map_x < maze_width and 0 <= map_y < maze_height):
                current_dist_wall = max_ray_distance
                break

            if maze[map_y, map_x] == 1:
                break
        
        dist_walls[i] = current_dist_wall

        # --- 2. Detecção de Agentes (Ray-Circle Intersection) ---
        for j in range(other_agent_data.shape[0]): # Use range
            if other_agent_data[j, 3] == 1: # Apenas detecta agentes vivos
                agent_center_x, agent_center_y = other_agent_data[j, 0], other_agent_data[j, 1]
                agent_radius = other_agent_data[j, 2] / 2 # Dividir por 2 para obter o raio do AGENT_SIZE (que é a largura)
                
                dist = _intersect_ray_circle_python(
                    agent_x, agent_y, ray_dir_x, ray_dir_y,
                    agent_center_x, agent_center_y, agent_radius
                )
                if dist != -1.0 and dist < dist_agents[i]:
                    dist_agents[i] = dist

        # --- 3. Detecção de Comida (Ray-Circle Intersection) ---
        for j in range(food_data.shape[0]): # Use range
            if food_data[j, 3] == 0: # Apenas detecta comida que não foi comida (is_eaten == 0)
                food_center_x, food_center_y = food_data[j, 0], food_data[j, 1]
                food_radius = food_data[j, 2] # food_data[j, 2] já é o raio da comida
                
                dist = _intersect_ray_circle_python(
                    agent_x, agent_y, ray_dir_x, ray_dir_y,
                    food_center_x, food_center_y, food_radius
                )
                if dist != -1.0 and dist < dist_food[i]:
                    dist_food[i] = dist
    
    return dist_walls, dist_agents, dist_food


# --- 3. CLASSE FOOD ---
class Food:
    def __init__(self, x: float, y: float):
        self.x = x
        self.y = y
        self.size = 8
        self.is_eaten = False
        # Superfície para desenho com transparência da fruta
        self.transparent_surface = pygame.Surface((self.size * 2, self.size * 2), pygame.SRCALPHA)

    def draw(self, screen: pygame.Surface):
        if not self.is_eaten:
            self.transparent_surface.fill((0, 0, 0, 0)) # Limpa com cor transparente
            local_center = pygame.math.Vector2(self.size, self.size)
            pygame.draw.circle(self.transparent_surface, COLOR_FOOD + (150,), local_center, self.size) # Alpha 150
            screen.blit(self.transparent_surface, (int(self.x - self.size), int(self.y - self.size)))

# --- 4. CLASSE AGENT (BASE) ---
class Agent:
    def __init__(self, x: float, y: float, angle: float = 0.0, color: Tuple[int, int, int] = COLOR_AI, num_rays: int = 9, fov: float = np.pi):
        self.x = x
        self.y = y
        self.angle = angle
        self.color = color # Cor atual (pode mudar para cinza ao morrer)
        
        self.num_rays = num_rays
        self.delta_theta = fov / (num_rays - 1) if num_rays > 1 else 0
        self.start_angle_offset = -fov / 2

        self.max_health = MAX_HEALTH
        self.health = float(MAX_HEALTH) # Vida como float para decaimento suave
        self.is_alive = True
        self.lifespan = 0.0 # Tempo que o agente viveu

        # Superfície para desenho semi-transparente do agente
        self.agent_draw_surface = pygame.Surface((AGENT_SIZE * 2, AGENT_SIZE * 2), pygame.SRCALPHA)

    def _check_collision(self, x: float, y: float) -> bool:
        """Verifica se uma posição colide com uma parede do labirinto."""
        return _check_collision_python(x, y, MAZE, CELL_SIZE, MAZE_WIDTH, MAZE_HEIGHT)

    def rotate(self, rotation: float):
        """Gira o agente em um determinado ângulo."""
        if self.is_alive:
            self.angle += rotation
            self.angle %= (2 * math.pi) # Mantém o ângulo entre 0 e 2*pi

    def move_forward(self, speed: float):
        """Move o agente para frente ou para trás."""
        if not self.is_alive:
            return

        # Calcular o ponto central do agente após o movimento
        new_x_center = self.x + math.cos(self.angle) * speed
        new_y_center = self.y + math.sin(self.angle) * speed

        # Verificar colisão com as 4 quinas de um quadrado delimitador do agente
        half_size = AGENT_SIZE / 2 
        
        # Considerando a rotação para os pontos de colisão para ser mais preciso
        # Offset das quinas em relação ao centro (ANTES da rotação)
        offsets = [
            pygame.math.Vector2(-half_size, -half_size),
            pygame.math.Vector2( half_size, -half_size),
            pygame.math.Vector2(-half_size,  half_size),
            pygame.math.Vector2( half_size,  half_size),
        ]
        
        collision_detected = False
        for offset in offsets:
            rotated_offset = offset.rotate_rad(self.angle)
            check_x = new_x_center + rotated_offset.x
            check_y = new_y_center + rotated_offset.y
            if self._check_collision(check_x, check_y):
                collision_detected = True
                break

        if not collision_detected:
            self.x = new_x_center
            self.y = new_y_center
            
    def update_health(self, dt: float):
        """Atualiza a vida do agente e seu tempo de vida."""
        if self.is_alive:
            self.lifespan += dt
            self.health -= HEALTH_DECAY_RATE * dt 
            self.health = max(0.0, self.health)
            if self.health <= 0:
                self.is_alive = False
                self.color = COLOR_DEAD_AGENT # Cor para agentes mortos já possui alpha

    def eat_food(self, amount: float):
        """Aumenta a vida do agente ao comer."""
        if self.is_alive:
            self.health += amount
            self.health = min(self.max_health, self.health)
            
    def _cast_rays_python(self, other_agent_data: np.ndarray, food_data: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
        """Lança raios de visão usando a função Python otimizada."""
        return _cast_rays_core_python(
            self.x, self.y, self.angle, self.num_rays, self.delta_theta, self.start_angle_offset,
            MAZE, CELL_SIZE, MAZE_WIDTH, MAZE_HEIGHT, MAX_RAY_DISTANCE,
            other_agent_data, food_data
        )

    def draw(self, screen: pygame.Surface, draw_rays: bool, other_agents_data: Optional[np.ndarray], food_data: Optional[np.ndarray]):
        """Desenha o agente, sua barra de vida e opcionalmente seus raios de visão."""
        # --- Desenho do Agente (triângulo) ---
        p1 = pygame.math.Vector2(AGENT_SIZE, 0)
        p2 = pygame.math.Vector2(-AGENT_SIZE / 2, -AGENT_SIZE / 2)
        p3 = pygame.math.Vector2(-AGENT_SIZE / 2, AGENT_SIZE / 2)
        
        p1_rot = p1.rotate_rad(self.angle)
        p2_rot = p2.rotate_rad(self.angle)
        p3_rot = p3.rotate_rad(self.angle)
        
        self.agent_draw_surface.fill((0, 0, 0, 0)) # Limpa a superfície com cor transparente
        
        local_center_offset = pygame.math.Vector2(AGENT_SIZE, AGENT_SIZE)
        local_points = [
            local_center_offset + p1_rot,
            local_center_offset + p2_rot,
            local_center_offset + p3_rot,
        ]
        
        # Desenha na superfície local com transparência (alpha 100 se vivo, ou cor morta com alpha)
        # self.color[:3] para obter RGB da cor base do agente.
        pygame.draw.polygon(self.agent_draw_surface, self.color[:3] + (100,) if self.is_alive else self.color, local_points)
        
        screen.blit(self.agent_draw_surface, (int(self.x - AGENT_SIZE), int(self.y - AGENT_SIZE)))

        # --- Desenho da Barra de Vida ---
        health_percentage = self.health / self.max_health
        health_bar_current_width = HEALTH_BAR_WIDTH * health_percentage
        
        bar_x = int(self.x - HEALTH_BAR_WIDTH / 2)
        bar_y = int(self.y + HEALTH_BAR_OFFSET_Y)

        pygame.draw.rect(screen, COLOR_HEALTH_BAR_BG, (bar_x, bar_y, HEALTH_BAR_WIDTH, HEALTH_BAR_HEIGHT), 0) # Fundo da barra
        
        health_color = COLOR_HEALTH_BAR_FULL if health_percentage > 0.3 else COLOR_HEALTH_BAR_LOW
        pygame.draw.rect(screen, health_color, (bar_x, bar_y, int(health_bar_current_width), HEALTH_BAR_HEIGHT), 0) # Preenchimento
        
        pygame.draw.rect(screen, (255, 255, 255), (bar_x, bar_y, HEALTH_BAR_WIDTH, HEALTH_BAR_HEIGHT), 1) # Borda da barra


        # --- Desenho dos Raios ---
        if draw_rays and self.is_alive and other_agents_data is not None and food_data is not None:
            dist_walls, dist_agents, dist_food = self._cast_rays_python(other_agents_data, food_data)

            ray_angles = [self.angle + self.start_angle_offset + i * self.delta_theta for i in range(self.num_rays)]
            
            ray_surface = pygame.Surface((SCREEN_WIDTH, SCREEN_HEIGHT), pygame.SRCALPHA)
            
            for i, angle in enumerate(ray_angles):
                final_dist = MAX_RAY_DISTANCE
                ray_color = COLOR_RAY_WALL 

                if dist_food[i] < final_dist:
                    final_dist = dist_food[i]
                    ray_color = COLOR_RAY_FOOD
                if dist_agents[i] < final_dist:
                    final_dist = dist_agents[i]
                    ray_color = COLOR_RAY_AGENT
                if dist_walls[i] < final_dist:
                    final_dist = dist_walls[i]
                    ray_color = COLOR_RAY_WALL
                
                end_x = self.x + math.cos(angle) * final_dist
                end_y = self.y + math.sin(angle) * final_dist
                
                pygame.draw.line(ray_surface, ray_color, (int(self.x), int(self.y)), (int(end_x), int(end_y)), 1)
            
            screen.blit(ray_surface, (0,0))


# --- 5. CLASSE AGENT INTELIGENTE (com Rede Neural) ---
class IntelligentAgent(Agent):
    def __init__(self, nn_instance: NN, x: float, y: float, angle: float = 0.0, color: Tuple[int, int, int] = COLOR_AI, num_rays: int = 9, fov: float = np.pi/2):
        super().__init__(x, y, angle, color, num_rays, fov)
        self.nn = nn_instance # Armazena o objeto NN

        self.input_size = self.num_rays * 3 # 3 tipos de detecção (parede, agente, comida) por raio
        self.hidden_size = self.nn.hidden_units
        self.output_size = self.nn.outputs

        # Valida a arquitetura da NN
        if self.nn.inputs != self.input_size or self.nn.outputs != 2:
            raise ValueError(
                f"NN architecture mismatch for IntelligentAgent. Expected inputs={self.input_size}, outputs=2. "
                f"Got inputs={self.nn.inputs}, outputs={self.nn.outputs} for NN instance: {self.nn}"
            )

    def update(self, dt: float, other_agent_data: np.ndarray, food_data: np.ndarray):
        """
        Atualiza o estado do agente, incluindo a tomada de decisão pela NN.
        `other_agent_data` já deve excluir o próprio agente que está sendo atualizado.
        """
        super().update_health(dt) # Passa dt para a atualização de vida
        
        if not self.is_alive:
            return

        # Lançar raios para coletar informações do ambiente
        dist_walls, dist_agents, dist_food = self._cast_rays_python(other_agent_data, food_data)

        # Concatena as distâncias dos raios em um único array de entrada para a NN
        # E normaliza os inputs (distâncias) para serem entre 0 e 1 para a NN
        nn_input_raw = np.concatenate((dist_walls, dist_agents, dist_food)).astype(np.float32)
        nn_input_normalized = nn_input_raw / MAX_RAY_DISTANCE
        
        # Faz a predição usando a função predict da classe NN
        # Ensure input is 2D (1, num_features)
        nn_output = self.nn.predict(nn_input_normalized.reshape(1, -1))[0] 
        
        # As saídas da NN são no intervalo [-1, 1] devido ao tanh na camada de saída
        rotation_output = nn_output[0] # Primeira saída para rotação
        move_output = (nn_output[1] + 1) / 2 # Segunda saída para movimento, remapeada de [-1,1] para [0,1]
                                             # para que 0 signifique parado e 1 signifique velocidade máxima.

        self.rotate(rotation_output * AGENT_ROTATION_SPEED)
        self.move_forward(move_output * AGENT_SPEED)


# --- Funções Auxiliares para Geração Aleatória ---
def get_random_valid_position(maze: np.ndarray, cell_size: int, existing_coords: Optional[List[Tuple[float, float]]] = None, min_distance: float = 0) -> Tuple[Optional[float], Optional[float]]:
    """
    Retorna uma tupla (x, y) de coordenadas de pixel aleatórias em um local válido (não parede),
    mantendo uma distância mínima de outras coordenadas existentes.
    """
    if existing_coords is None:
        existing_coords = []

    MAZE_HEIGHT, MAZE_WIDTH = maze.shape
    attempts = 0
    max_attempts = 1000 # Limite de tentativas para evitar loops infinitos em mapas lotados
    while attempts < max_attempts:
        grid_x = random.randint(0, MAZE_WIDTH - 1)
        grid_y = random.randint(0, MAZE_HEIGHT - 1)

        if maze[grid_y, grid_x] == 0: # Se a célula não é parede
            px = grid_x * cell_size + cell_size / 2
            py = grid_y * cell_size + cell_size / 2
            
            too_close = False
            for ex, ey in existing_coords:
                if math.hypot(px - ex, py - ey) < min_distance:
                    too_close = True
                    break
            
            if not too_close:
                return px, py
        attempts += 1
    return None, None # Não encontrou posição válida após várias tentativas

def generate_intelligent_agents(neural_networks: List[NN], maze: np.ndarray, cell_size: int) -> List[IntelligentAgent]:
    """Gera uma lista de IntelligentAgents, um para cada NN fornecida."""
    agents = []
    existing_positions = [] # Para evitar que agentes nasçam uns sobre os outros

    for nn_instance in neural_networks:
        x, y = get_random_valid_position(maze, cell_size, existing_positions, AGENT_SIZE * 2)
        if x is None:
            print(f"Aviso: Não foi possível posicionar todos os agentes. {len(agents)} de {len(neural_networks)} solicitados foram colocados.")
            break # Interrompe se não houver mais posições válidas
        
        angle = random.uniform(0, 2 * np.pi)
        # Cor randomizada para cada agente
        random_color = (random.randint(100, 255), random.randint(100, 255), random.randint(100, 255))
        
        agent = IntelligentAgent(nn_instance, x, y, angle=angle, color=random_color)
        agents.append(agent)
        existing_positions.append((x, y)) # Adiciona a posição do novo agente
    
    return agents

def generate_random_food(num_food: int, maze: np.ndarray, cell_size: int, existing_object_coords: Optional[List[Tuple[float, float]]] = None) -> List[Food]:
    """Gera uma lista de itens de comida em posições aleatórias válidas."""
    food_items = []
    
    if existing_object_coords is None:
        existing_object_coords = []
    
    for _ in range(num_food):
        x, y = get_random_valid_position(maze, cell_size, existing_object_coords, AGENT_SIZE + Food(0,0).size)
        if x is None:
            print(f"Aviso: Não foi possível posicionar todos os itens de comida. {len(food_items)} de {num_food} solicitados foram colocados.")
            break
        food_items.append(Food(x, y))
        existing_object_coords.append((x, y)) # Adiciona a posição da comida para evitar sobreposição
    return food_items


# ==============================================================================
# 6. FUNÇÃO PRINCIPAL DA SIMULAÇÃO
# ==============================================================================

def run_game_simulation(
    neural_networks: List[NN],
    world_size: Tuple[int, int] = (SCREEN_WIDTH, SCREEN_HEIGHT),
    num_fruits: int = NUM_INITIAL_FOOD,
    render_percentage: float = 0.1,
    skip_frames: int = 0, # -1 para não renderizar nada
    initial_render_rays: bool = True # Estado inicial para renderização de raios
) -> List[Tuple[np.ndarray, float]]:
    """
    Roda a simulação do jogo com múltiplos agentes inteligentes no mesmo mapa.

    Args:
        neural_networks (List[NN]): Uma lista de redes neurais, uma para cada agente.
        world_size (Tuple[int, int]): O tamanho (largura, altura) da janela de visualização.
        num_fruits (int): Quantidade de frutas no mapa.
        render_percentage (float): A porcentagem de agentes a serem renderizados (0.0 a 1.0).
        skip_frames (int): Quantidade de quadros a pular entre renderizações. -1 para não renderizar nada.
        initial_render_rays (bool): Se os raios de visão devem ser renderizados por padrão.

    Returns:
        List[Tuple[np.ndarray, float]]: Uma lista de tuplas contendo
                                        (parâmetros achatados da NN do agente, tempo de vida em segundos).
    """
    # Inicializa Pygame apenas se a renderização estiver ativa
    if skip_frames != -1:
        pygame.init()
        screen = pygame.display.set_mode(world_size)
        pygame.display.set_caption("Simulação de Agentes Inteligentes no Labirinto")
        font = pygame.font.SysFont(None, 24)
        print("\n--- Controles de Visualização ---")
        print("Tecla 'R': Ligar/Desligar visualização dos raios")
        print("Tecla 'ESPAÇO': Alternar conjunto de agentes renderizados")
        print("--------------------------------\n")
    
    clock = pygame.time.Clock()

    num_total_agents = len(neural_networks)
    print(f"Iniciando simulação com {num_total_agents} agentes.")
    
    # Gera agentes inteligentes e comida
    agents = generate_intelligent_agents(neural_networks, MAZE, CELL_SIZE)
    if not agents: # Caso nenhum agente possa ser posicionado
        print("Nenhum agente pôde ser posicionado no mapa. Encerrando simulação.")
        if skip_frames != -1: pygame.quit()
        return []

    # Coleta todas as posições iniciais de agentes para evitar sobreposição com comida
    all_current_object_coords = [(a.x, a.y) for a in agents]
    food_items = generate_random_food(num_fruits, MAZE, CELL_SIZE, all_current_object_coords)

    # Configuração inicial para renderização
    num_to_render = int(num_total_agents * render_percentage)
    indices_to_render = []
    if num_to_render > 0:
        # Seleção inicial de agentes para renderizar, priorizando vivos
        alive_agent_indices = [i for i, agent in enumerate(agents) if agent.is_alive]
        if len(alive_agent_indices) > 0:
            indices_to_render = random.sample(alive_agent_indices, k=min(len(alive_agent_indices), num_to_render))
        else:
            indices_to_render = [] # Nenhum agente vivo para renderizar inicialmente
    print(f"Renderizando {len(indices_to_render)} agentes.")
    
    draw_rays_flag = initial_render_rays # Flag para controlar a visualização dos raios
    
    # Lista para armazenar (parâmetros da NN achatados, tempo de vida)
    agent_results = []
    
    running = True
    frame_count = 0 # Contador de quadros
    
    while running:
        # --- Tratamento de Eventos (apenas se a renderização estiver ativa) ---
        if skip_frames != -1:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_r: # Alternar visualização dos raios
                        draw_rays_flag = not draw_rays_flag
                        print(f"Visualização de raios: {'ON' if draw_rays_flag else 'OFF'}")
                    if event.key == pygame.K_SPACE: # Alternar conjunto de agentes renderizados
                        alive_agent_indices = [i for i, agent in enumerate(agents) if agent.is_alive]
                        if len(alive_agent_indices) > 0:
                            num_to_sample = min(len(alive_agent_indices), num_to_render)
                            indices_to_render = random.sample(alive_agent_indices, k=num_to_sample)
                            print(f"Alternando para renderizar {len(indices_to_render)} novos agentes.")
                        else:
                            indices_to_render = []
                            print("Nenhum agente vivo para renderizar.")

        # --- Lógica de Atualização do Jogo (fixa em 60 FPS) ---
        dt = clock.tick(60) / 1000.0 # Delta time em segundos (ex: 1/60 = 0.0166)
        if dt == 0: dt = 1/60.0 # Previne dt zero, caso clock.tick seja muito rápido ou pausado

        active_agents_count = 0
        
        # Prepara os dados de todos os agentes e comida para as funções de raycasting e colisão
        all_agent_data_for_raycast = np.array([
            [a.x, a.y, AGENT_SIZE, 1 if a.is_alive else 0]
            for a in agents
        ], dtype=np.float32)

        food_data_for_raycast = np.array([
            [f.x, f.y, f.size, 1 if f.is_eaten else 0] for f in food_items
        ], dtype=np.float32)

        # Itera sobre os agentes para atualização
        for idx, agent in enumerate(agents):
            if agent.is_alive:
                active_agents_count += 1
                
                # Cria uma cópia dos dados de outros agentes, excluindo o agente atual para o raycast
                # np.delete é usado aqui, que cria uma nova array. Para muitos agentes, isso pode ser lento.
                # Uma otimização seria passar o índice do agente atual e manipular dentro do loop de raycast,
                # mas isso requer reintroduzir a lógica de Numba ou um loop Python mais complexo no raycast.
                other_agent_data_for_current_agent_raycast = np.delete(all_agent_data_for_raycast, idx, axis=0)
                
                agent.update(dt, other_agent_data_for_current_agent_raycast, food_data_for_raycast)
            elif not hasattr(agent, '_lifespan_recorded'):
                # Registra o tempo de vida do agente que acabou de morrer
                agent_results.append((flatten_parameters(agent.nn), agent.lifespan))
                agent._lifespan_recorded = True # Marca como registrado para não duplicar

        # Verifica consumo de comida
        for agent in agents:
            if not agent.is_alive:
                continue # Agentes mortos não comem
            
            for food in food_items:
                if food.is_eaten:
                    continue

                distance = math.hypot(agent.x - food.x, agent.y - food.y)
                # O raio do agente AGENT_SIZE é na verdade AGENT_SIZE/2
                if distance < AGENT_SIZE / 2 + food.size: # Se a distância é menor que a soma dos raios
                    agent.eat_food(FOOD_HEAL_AMOUNT)
                    food.is_eaten = True
                    break # Agente só pode comer uma fruta por ciclo de atualização

        # Remove comida comida e reaparece novas
        food_items = [f for f in food_items if not f.is_eaten]
        
        # Coordenadas de objetos existentes para evitar spawn de comida em cima
        current_all_object_coords = [(a.x,a.y) for a in agents if a.is_alive] + [(f.x,f.y) for f in food_items]
        
        while len(food_items) < num_fruits:
            x, y = get_random_valid_position(MAZE, CELL_SIZE, current_all_object_coords, AGENT_SIZE + Food(0,0).size)
            if x is not None:
                new_food = Food(x,y)
                food_items.append(new_food)
                current_all_object_coords.append((x,y)) # Adiciona a nova comida às coords existentes
            else:
                print("Aviso: Não foi possível posicionar todos os itens de comida desejados.")
                break # Sai do loop se não conseguir posicionar mais comida


        if active_agents_count == 0:
            print("Todos os agentes morreram. Fim da simulação.")
            running = False

        # --- Renderização (se permitida pelo skip_frames) ---
        should_render = (skip_frames != -1) and (frame_count % (skip_frames + 1) == 0)
        
        if should_render:
            screen.fill(COLOR_BG)

            # Desenha o chão e as paredes do labirinto
            for y in range(MAZE_HEIGHT):
                for x in range(MAZE_WIDTH):
                    color = COLOR_WALL if MAZE[y, x] == 1 else COLOR_FLOOR
                    pygame.draw.rect(screen, color, (x * CELL_SIZE, y * CELL_SIZE, CELL_SIZE, CELL_SIZE))

            # Desenha os itens de comida (já são semi-transparentes em seu método draw)
            for food in food_items:
                food.draw(screen)

            # Prepara os dados de agentes e comida para raycasting durante o desenho dos raios
            # (necessário porque os dados podem ter mudado desde a fase de update)
            all_agent_data_for_draw_rays = np.array([
                [a.x, a.y, AGENT_SIZE, 1 if a.is_alive else 0]
                for a in agents
            ], dtype=np.float32)
            food_data_for_draw_rays = np.array([
                [f.x, f.y, f.size, 1 if f.is_eaten else 0] for f in food_items
            ], dtype=np.float32)

            # Desenha apenas os agentes selecionados para renderização
            for i in indices_to_render:
                agent = agents[i]
                other_agent_data_for_draw_raycast = np.delete(all_agent_data_for_draw_rays, i, axis=0)
                agent.draw(screen, 
                           draw_rays=draw_rays_flag, 
                           other_agents_data=other_agent_data_for_draw_raycast,
                           food_data=food_data_for_draw_rays)
            
            # Exibe informações na tela
            info_text = f"Agentes Vivos: {active_agents_count}/{num_total_agents} | FPS: {clock.get_fps():.1f} | Raios: {'ON' if draw_rays_flag else 'OFF'}"
            text_surface = font.render(info_text, True, (255, 255, 255))
            screen.blit(text_surface, (10, 10))

            pygame.display.flip()

        frame_count += 1
    
    # --- Finalização da Simulação ---
    if skip_frames != -1:
        pygame.quit()
        
    # Adiciona os resultados dos agentes que ainda estavam vivos no final da simulação
    for agent in agents:
        if not hasattr(agent, '_lifespan_recorded'):
            agent_results.append((flatten_parameters(agent.nn), agent.lifespan))

    return agent_results

# ==============================================================================
# 7. BLOCO DE EXEMPLO DE USO
# ==============================================================================

if __name__ == '__main__':
    # --- Parâmetros da Simulação ---
    NUM_AGENTS = 10
    RENDER_PERCENT = 1  # Mostrar 20% dos agentes (10 agentes neste caso)
    SKIP_FRAMES_OPTION = 0 # 0 = renderizar todos os frames, 5 = pular 5 frames, -1 = não renderizar
    INITIAL_RENDER_RAYS = True # Renderizar raios por padrão

    # --- Criação das Redes Neurais ---
    # Cada agente precisa de (num_rays * 3) entradas e 2 saídas
    # Assumindo 9 raios como padrão na classe IntelligentAgent
    input_neurons = 9 * 3 # 9 raios * (dist_parede, dist_agente, dist_comida)
    hidden_neurons = 16 
    output_neurons = 2    # (girar, andar)
    
    agent_networks = [
        NN(inputs=input_neurons, hidden_units=hidden_neurons, outputs=output_neurons, activation='tanh')
        for _ in range(NUM_AGENTS)
    ]

    # --- Executar a Simulação ---
    results = run_game_simulation(
        neural_networks=agent_networks,
        num_fruits=NUM_INITIAL_FOOD, 
        render_percentage=RENDER_PERCENT,
        skip_frames=SKIP_FRAMES_OPTION,
        initial_render_rays=INITIAL_RENDER_RAYS
    )

    # --- Mostrar Resultados ---
    if results:
        # Extrai apenas os tempos de vida para estatísticas
        lifespans = [r[1] for r in results] 
        print("\n--- Resultados da Simulação ---")
        print(f"Total de agentes simulados: {NUM_AGENTS}")
        print(f"Total de resultados retornados (agentes que morreram ou terminaram): {len(lifespans)}")
        
        if lifespans: # Verifica se há tempos de vida para calcular estatísticas
            print(f"Tempo de vida médio: {np.mean(lifespans):.2f} segundos")
            print(f"Melhor tempo de vida: {np.max(lifespans):.2f} segundos")
            print(f"Pior tempo de vida: {np.min(lifespans):.2f} segundos")

            # Exemplo de como acessar os parâmetros de uma NN específica (ex: a melhor)
            best_agent_idx = np.argmax(lifespans)
            best_nn_params = results[best_agent_idx][0]
            print(f"\nParâmetros da NN do agente com melhor tempo de vida (top 5 elementos): {best_nn_params[:5]}...")
        else:
            print("Nenhum agente sobreviveu o suficiente para ter um tempo de vida registrado.")


--- Controles de Visualização ---
Tecla 'R': Ligar/Desligar visualização dos raios
Tecla 'ESPAÇO': Alternar conjunto de agentes renderizados
--------------------------------

Iniciando simulação com 10 agentes.
Renderizando 10 agentes.

--- Resultados da Simulação ---
Total de agentes simulados: 10
Total de resultados retornados (agentes que morreram ou terminaram): 10
Tempo de vida médio: 46.41 segundos
Melhor tempo de vida: 46.41 segundos
Pior tempo de vida: 46.41 segundos

Parâmetros da NN do agente com melhor tempo de vida (top 5 elementos): [ 0.04787694  0.25427473  0.186621   -0.18516508  0.17714386]...
